### 采用预训练方式 进行训练

## 1 准备数据

In [1]:
import torch
from torch.utils.data import Dataset
import h5py

import os    # 用来加载数据
import shutil     # 用来移动图片
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**将下载好的文件放到对应的目录**

train.zip  里面

In [ ]:
# 创建 猫狗文件夹 (注意运行一次就行)
train_root = './kaggle/train/'
dog_folder = os.path.join(train_root, 'dog')
cat_folder = os.path.join(train_root, 'cat')
os.mkdir(dog_folder)
os.mkdir(cat_folder)

val_root = './kaggle/val'
dog_folder = os.path.join(val_root, 'dog')
cat_folder = os.path.join(val_root, 'cat')
os.mkdir(dog_folder)
os.mkdir(cat_folder)

In [ ]:
# 将猫和狗的图片分别移到对应的文件夹里
data_file = os.listdir('./data/zip/')

dog_file = list(filter(lambda x:x[:3]=='dog', data_file))  # filter(function, iterable)
cat_file = list(filter(lambda x:x[:3]=='cat', data_file))

root = './data/'
# 移动狗的图片
for i in range(len(dog_file)):
    pic_path = root + 'zip/' + dog_file[i]
    if i < len(dog_file)*0.9:
        obj_path = train_root + '/dog/' + dog_file[i]
    else:
        obj_path = val_root + '/dog/' + dog_file[i]
    shutil.move(pic_path, obj_path)     # 移动图片
# 移动猫的图片    
for i in range(len(cat_file)):
    pic_path = root + 'zip/' + cat_file[i]
    if i < len(cat_file)*0.9:
        obj_path = train_root + '/cat/' + cat_file[i]
    else:
        obj_path = val_root + '/cat/' + cat_file[i]
    shutil.move(pic_path, obj_path)

In [ ]:
dogfiles = os.listdir('kaggle/train/dog/')
catfiles = os.listdir('kaggle/train/cat/')

dogfile = list(filter(lambda x:x[:3]=='dog', dogfiles))
catfile = list(filter(lambda x:x[:3]=='cat', catfiles))

# 从拆分的数据集中随机取一张图片
dog_show = np.random.choice(dogfile, size=1)      # (dataset, size, replace, p)  size表示抽取多少个
cat_show = np.random.choice(catfile, size=1)

dog_path_show = plt.imread('kaggle/train/dog/' + dog_show[0])  # 注意 不能为 dog_show  得为 dog_show[0]
cat_path_show = plt.imread('kaggle/train/cat/' + cat_show[0])  # 注意不能为 cat_show  得为 cat_show[0]

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(dog_path_show)
ax2.imshow(cat_path_show)

## 2 采用预训练方式

- resnet18  (通过设置fix_param 来控制是否需要进行卷积层参数的更新)
- vgg19,inceptionv3,resnet152组合的预训练网络

In [2]:
import os
import time

import torch
from torchvision import models, transforms
from torch import optim, nn
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [3]:
# 数据增强
data_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(299),  # transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2)
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))        
    ]),
    'val':
    transforms.Compose([
        transforms.Resize(320),    # 将原来的Scale 替换为Resize
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])
}

In [7]:
# 使用ImageFolder定义数据文件夹以从文件夹中获取图像和类
root = 'kaggle/'
data_folder = {
    'train':
    ImageFolder(
        os.path.join(root, 'train'), transform=data_transforms['train']),
    'val':
    ImageFolder(
        os.path.join(root, 'val'), transform=data_transforms['val'])
}

# 定义dataloader  加载数据
batch_size = 32
dataloader = {
    'train':
    DataLoader(
        data_folder['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=4),
    'val':
    DataLoader(
        data_folder['val'],
        batch_size=batch_size,
        num_workers=4)
}

# 获取训练集集和验证集的大小
data_size = {
    'train': len(dataloader['train'].dataset),
    'val': len(dataloader['val'].dataset)
}

# 获取类别数
img_classes = len(dataloader['train'].dataset.classes)

#device : GPU or CPU
use_gpu = torch.cuda.is_available()   #  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

fix_param = True    # 是否固定卷积层参数

# 定义model
transfer_model = models.resnet18(pretrained=True)


if fix_param:
    for param in transfer_model.parameters():
        param.requires_grad = False
dim_in = transfer_model.fc.in_features
transfer_model.fc = nn.Linear(dim_in, img_classes)    # 此时这里img_classes=2为猫和狗两类

if use_gpu:
    transfer_model = transfer_model.cuda()

# 如果固定卷积层参数，只需要更新全连接层参数即可， 否则需要全部更新
if fix_param:
    optimizer = optim.Adam(transfer_model.fc.parameters(), lr=1e-3)
else:
    optimizer = optim.Adam(transfer_model.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss()

# train
num_epoch = 10

for epoch in range(num_epoch):
    print('{}/{}'.format(epoch+1, num_epoch))
    print('*'*20+' train ' + '*'*20)
    transfer_model.train()
    running_loss = 0.0
    running_acc = 0.0
    since = time.time()
    
    for i, data in enumerate(dataloader['train'], 1):
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        img = Variable(img)
        label = Variable(label)

        # forward
        out = transfer_model(img)
        loss = criterion(out, label)
        _, pred = torch.max(out, 1)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * label.size(0)
        num_correct = (pred == label).sum().item()
        running_acc += num_correct
        if i % 100 == 0:
            print('Loss: {:.6f}, Acc: {:.4f}'.format(running_loss / (
                i * batch_size), running_acc / (i * batch_size)))
            
    running_loss /= data_size['train']
    running_acc /= data_size['train']
    elips_time = time.time() - since
    print('Loss: {:.6f}, Acc: {:.4f}, Time: {:.0f}s'.format(
        running_loss, running_acc, elips_time))
    
    print('Validation')
    transfer_model.eval()
    num_correct=0.0
    total = 0.0
    eval_loss = 0.0
    for data in dataloader['val']:
        img, label = data
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        out = transfer_model(img)
        _, pred = out.max(1)
        loss = criterion(out, label)
        eval_loss += loss.item() * label.size(0)
        num_correct += (pred == label).sum().item()
        total += label.size(0)
        
    print('Loss: {:.6f}   Acc:{:.4f}'.format(eval_loss/total, num_correct/total))
    
print('Finish Training!')

1/10
******************** train ********************
Loss: 0.402148, Acc: 0.8263
Loss: 0.322686, Acc: 0.8667
Loss: 0.285819, Acc: 0.8834
Loss: 0.267471, Acc: 0.8896
Loss: 0.251834, Acc: 0.8948
Loss: 0.240083, Acc: 0.8997
Loss: 0.236034, Acc: 0.9008
Loss: 0.235792, Acc: 0.9008, Time: 378s
Validation
Loss: 0.067775   Acc:0.9752
2/10
******************** train ********************
Loss: 0.183648, Acc: 0.9222
Loss: 0.186092, Acc: 0.9189
Loss: 0.183786, Acc: 0.9196
Loss: 0.195190, Acc: 0.9134
Loss: 0.192634, Acc: 0.9147
Loss: 0.190080, Acc: 0.9158
Loss: 0.190055, Acc: 0.9148
Loss: 0.189941, Acc: 0.9149, Time: 90s
Validation
Loss: 0.061061   Acc:0.9764
3/10
******************** train ********************
Loss: 0.169788, Acc: 0.9206
Loss: 0.172517, Acc: 0.9225
Loss: 0.178542, Acc: 0.9205
Loss: 0.177521, Acc: 0.9208
Loss: 0.175774, Acc: 0.9213
Loss: 0.177689, Acc: 0.9214
Loss: 0.176160, Acc: 0.9221
Loss: 0.175990, Acc: 0.9223, Time: 91s
Validation
Loss: 0.068427   Acc:0.9728
4/10
*************

In [6]:
print(use_gpu)

True


**使用迁移学习，组合多个预训练网络，都将它们的卷积层参数固定，只更新最后的全连接层的参数，在每一次迭代中，都需要将图片前向传播，通过卷积层到全连接层，最后输出结果，接着进行反向传播更新全连接层的参数。如果数据集特别大，这样做效率特别低，因为卷积层的参数没有更新，所以每次迭代中数据集前向传播经过卷积层的结果是一样的，所以每次迭代中数据集前向传播经过卷积层的结果是一样的，所以没有必要每次都进行前向传播，只需要将数据集一次迭代中前向传播经过卷积网络的结果保存起来就可以了，这个结果成为特征向量**

多个预训练网络 不好在jupyter里炒作，写成.py文件在pycharm中运行